In [ ]:
#>>>>>>>>>>>>>>FINE TUNING BOTH RESNET50V2 AND LSTM SIMULTANEOUSLY<<<<<

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, TimeDistributed, LSTM, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical, Sequence
import numpy as np
import os
import random

# ================================
# 1️⃣ CONFIG
# ================================
DATASET_PATH = "/content/drive/MyDrive/skeleton_dataset"  # new/updated dataset
IMG_SIZE = 224
BATCH_SIZE = 4
TIMESTEPS = 16
NUM_CLASSES = 4
EPOCHS_CONTINUE = 5
EPOCHS_CONTINUE_2 = 5  # adjust based on dataset size

# Class mapping
class_indices = {'body':0, 'hand':1, 'head':2, 'normal':3}
class_names = list(class_indices.keys())

# ================================
# 2️⃣ CUSTOM SEQUENCE GENERATOR
# ================================
class SkeletonSequence(Sequence):
    def __init__(self, folder_path, timesteps=TIMESTEPS, batch_size=BATCH_SIZE, shuffle=True):
        self.folder_path = folder_path
        self.timesteps = timesteps
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.samples = []

        for class_name, idx in class_indices.items():
            class_folder = os.path.join(folder_path, class_name)
            images = sorted(os.listdir(class_folder))
            random.shuffle(images)  # shuffle images to avoid order bias

            # sliding window with stride=1 to maximize data usage
        stride = TIMESTEPS  # or 1 if you want maximum sequences
        for i in range(0, len(images) - timesteps + 1, stride):
            seq_files = images[i:i+timesteps]
            self.samples.append((seq_files, class_name))


        if shuffle:
            random.shuffle(self.samples)

    def __len__(self):
        return int(np.ceil(len(self.samples) / self.batch_size))

    def __getitem__(self, idx):
        batch_samples = self.samples[idx*self.batch_size:(idx+1)*self.batch_size]
        X = np.zeros((len(batch_samples), self.timesteps, IMG_SIZE, IMG_SIZE, 3), dtype=np.float32)
        y = np.zeros((len(batch_samples), NUM_CLASSES), dtype=np.float32)

        for i, (seq_files, class_name) in enumerate(batch_samples):
            for t, fname in enumerate(seq_files):
                img_path = os.path.join(self.folder_path, class_name, fname)
                img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
                img_array = img_to_array(img)
                img_array = tf.keras.applications.resnet_v2.preprocess_input(img_array)
                X[i, t] = img_array

            y[i] = to_categorical(class_indices[class_name], NUM_CLASSES)

        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.samples)

# ================================
# 3️⃣ LOAD TRAIN/VAL GENERATORS
# ================================
all_seq = SkeletonSequence(DATASET_PATH, shuffle=True)
split = int(0.8 * len(all_seq.samples))

train_gen = SkeletonSequence(DATASET_PATH, shuffle=True)
train_gen.samples = all_seq.samples[:split]

val_gen = SkeletonSequence(DATASET_PATH, shuffle=False)
val_gen.samples = all_seq.samples[split:]

print(f"Train sequences: {len(train_gen.samples)}, Val sequences: {len(val_gen.samples)}")

# ================================
# 4️⃣ LOAD PRETRAINED MODELS
# ================================
# Load ResNet50V2
cnn = load_model("/content/drive/MyDrive/resnet50v2_best.keras")


In [ ]:
# ================================
# Stage 1: Fine-tune last 10 layers of CNN
# ================================

# Freeze all layers except last 10
for layer in cnn.layers[:-10]:
    layer.trainable = False
for layer in cnn.layers[-10:]:
    layer.trainable = True

# Compile model for Stage 1
end_to_end_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),  # slightly higher LR for head layers
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks_stage1 = [
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss', verbose=1),
    tf.keras.callbacks.ModelCheckpoint(
        "/content/drive/MyDrive/resnet50v2_lstm_stage1.keras",
        save_best_only=True,
        monitor='val_loss',
        verbose=1
    )
]

# Train Stage 1
history_stage1 = end_to_end_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_CONTINUE,
    batch_size=BATCH_SIZE,
    callbacks=callbacks_stage1
)


In [ ]:
# ================================
# Final Cell: Combine Stage 1 + Stage 2 histories
# ================================
import matplotlib.pyplot as plt

# Combine Stage 1 and Stage 2 metrics
loss = history_stage1.history['loss'] + history_stage2.history['loss']
val_loss = history_stage1.history['val_loss'] + history_stage2.history['val_loss']
accuracy = history_stage1.history['accuracy'] + history_stage2.history['accuracy']
val_accuracy = history_stage1.history['val_accuracy'] + history_stage2.history['val_accuracy']

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
axes[0].plot(loss, label='Train Loss', marker='o', linewidth=2)
axes[0].plot(val_loss, label='Val Loss', marker='s', linewidth=2)
axes[0].set_title('Fine-tuning Loss (Stage1+Stage2)')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(accuracy, label='Train Accuracy', marker='o', linewidth=2)
axes[1].plot(val_accuracy, label='Val Accuracy', marker='s', linewidth=2)
axes[1].set_title('Fine-tuning Accuracy (Stage1+Stage2)')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/fine_tuning_combined_history.png', dpi=150, bbox_inches='tight')
plt.show()
